In [ ]:
import pandas as pd
import numpy as np
import os
import sys

#Deze staan ook in rdkithelperdinges
# noinspection PyPackageRequirements
import rdkit

# noinspection PyPackageRequirements
import rdkit.Chem.AllChem

# noinspection PyPackageRequirements
import rdkit.Chem.Draw

# noinspection PyPackageRequirements
import rdkit.Chem.rdmolfiles



# Load the data and couple the SMILES to the yields and remove nan's

In [ ]:
db_dir_smiles = "data/compounds_smiles.csv"
db_dir_yields = "data/compounds_yield.csv"

db_smiles = pd.read_csv(db_dir_smiles)
db_yields = pd.read_csv(db_dir_yields)




Convert the SMILES to Graphs